In [1]:
from Dissects.io import (load_NDskl, 
                         load_image,
                         load_skeleton,
                         save_skeleton,
                         save_fits,
                         save_image,
                        )
from Dissects.image import (z_project,
                            thinning,
                            dilation)
from Dissects.geometry import Skeleton


from Dissects.analysis.analysis import (general_analysis,
                              cellstats
                              )

from Dissects.analysis.analysis_3D_apical import (junction_intensity,
                                                  morphology_analysis)

from Dissects.segmentation.seg_3D_apical import (generate_segmentation)


In [2]:
import os

from skimage import io
import numpy as np
import pandas as pd
import copy
import sys
from skimage import morphology

import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.colors import ListedColormap
import plotly.graph_objects as go

sys.setrecursionlimit(10000)

import plotly.express as px

from scipy.ndimage.morphology import (binary_fill_holes,
                                      binary_dilation,
                                     )
from scipy import ndimage
import scipy as sci
# %matplotlib notebook
%load_ext snakeviz

In [117]:
directory="/media/admin-suz/Datas/Comparative-analysis_Manu-Tatiana/Squelettes_Disperse"

filename="T2-T3_20210429_sqhRFPt3B-EcadGFP_leg_FastAiSc-Opt1160x1160_pixelsize0v09_Speed7_GB2.fits_c1.35e+03.up.NDskl.a.NDskl"

In [118]:
# get original image
img0 = load_image(os.path.join(directory, 
                              "T2-T3_20210429_sqhRFPt3B-EcadGFP_leg_FastAiSc-Opt1160x1160_pixelsize0,09_Speed7.tif"))

In [119]:
cp, fil, point, specs = load_NDskl(os.path.join(directory, filename))

# #create skeleton object
skel = Skeleton(cp, fil, point, specs)

/home/admin-suz/miniconda3/lib/python3.8/site-packages/Dissects-0.0.1-py3.8.egg/Dissects/geometry/skeleton.py:24: UserWarning:

Has pixel can be half pixel from DisPerSE output,                   we take the integer part of xyz values. 



In [120]:
#Clean skeleton
skel.remove_lonely_cp()
skel.remove_free_filament()

/home/admin-suz/miniconda3/lib/python3.8/site-packages/Dissects-0.0.1-py3.8.egg/Dissects/geometry/skeleton.py:60: UserWarning:

specs dictionnary is not updated, ncrit value is not updated.



In [121]:
skel.specs

{'ndims': 3,
 'bbox': array([0., 0., 0.]),
 'bbox_delta': array([ 166., 1052.,  106.]),
 'ncrit': 1811,
 'nfil': 1666}

In [122]:
skeleton_image = skel.create_binary_image()
img_binary_3d = morphology.skeletonize(skeleton_image)

# Global variable

Ce n'est pas idéal, mais c'est une solution temporaire pour choisir tout les paramètres au début de l'analyse sans avoir à les chercher dans les différentes cellules...

In [123]:
# Pixel/Voxel size
X_SIZE = 0.0916155
Y_SIZE = 0.0916155
Z_SIZE = 0.2201818

# Save face, edge, vert dataframe from initial segmentation
SAVE_INIT_SEGMENTATION = False
# Save face, edge, vert image from initial segmentation
SAVE_INIT_SEGMENTATION_IMAGE = False

DIRECT_VISUALISATION_VERTEX_EDGES = True
DIRECT_VISUALISATION_FACE = True

# Analyse morphologique
MORPHOLOGICAL_ANALYSIS = True
AREA = True
PERIMETER = True
NB_NEIGHBOR = True
ANISO = True

In [124]:
X_SHAPE = img0[0].shape[2]
Y_SHAPE = img0[0].shape[1]
Z_SHAPE = img0[0].shape[0]


# 3D vertex detection

In [125]:
%pdb

Automatic pdb calling has been turned ON


In [126]:
face_df, edge_df, vert_df, points_df = generate_segmentation(skel, clean=False, **{"X_SIZE":X_SIZE,
                                                                                   "Y_SIZE":Y_SIZE,
                                                                                   "Z_SIZE":Z_SIZE})

if SAVE_INIT_SEGMENTATION : 
    vert_df.to_csv(os.path.join(directory, 'vert_df.csv'))
    edge_df.to_csv(os.path.join(directory, 'edge_df.csv'))
    face_df.to_csv(os.path.join(directory, 'face_df.csv'))


there is a problem
there is a problem
there is a problem
there is a problem


In [127]:
if SAVE_INIT_SEGMENTATION_IMAGE :
    
    # Vertex image
    vertex_image = np.zeros([Z_SHAPE, Y_SHAPE, X_SHAPE])
    s = ndimage.generate_binary_structure(3,3)

    vertex_image[vert_df.z.to_numpy(),
                 vert_df.y.to_numpy(),
                 vert_df.x.to_numpy()] = 1
    # Dilate a little bit vertex to have a better viewing
    vertex_image = ndimage.morphology.binary_dilation(vertex_image, structure=s)
    
    save_image(vertex_image, 'output_vertex.tif', directory, **{"x_size":X_SIZE,
                                                                "y_size":Y_SIZE,
                                                                "z_size":Z_SIZE,})
    
    image_skeleton = skel.create_binary_image()
    image_skeleton = ndimage.morphology.binary_dilation(image_skeleton, structure=s)
    
    save_image(image_skeleton, 'output_skeleton.tif', directory, **{"x_size":X_SIZE,
                                                                    "y_size":Y_SIZE,
                                                                    "z_size":Z_SIZE,})

In [128]:
if DIRECT_VISUALISATION_VERTEX_EDGES: 
    # plot skeleton
    image_skeleton = skel.create_binary_image()
    z0, y0,x0 = np.where(image_skeleton>0)
    background_skeleton = go.Scatter3d(x=x0,
                                        y=y0,
                                        z=z0,
                                        mode='markers',
                                        marker=dict(
                                            size=2,
                                            color='black',
                                            opacity=0.1
                                         )
                        )
    
    
    vertex = go.Scatter3d( x=vert_df.x_pix,
                           y=vert_df.y_pix,
                           z=vert_df.z_pix,
                           mode='markers',
                           marker=dict(
                            size=2,
                            color='red', 
                            opacity=0.5
                         )
                )
    
    fig = go.Figure(data=[background_skeleton, vertex], )
    fig.update_layout(title='Background-vertex', 
                  autosize=False,
                  width=1000,
                  height=1000,
                  margin=dict(l=65, r=50, b=65, t=90),
                 )
    
    # Edges
    zs,ys,xs = vert_df.loc[edge_df['srce']][['z_pix', 'y_pix', 'x_pix']].values.flatten(order='F').reshape(3,edge_df.shape[0])
    zt,yt,xt = vert_df.loc[edge_df['trgt']][['z_pix', 'y_pix', 'x_pix']].values.flatten(order='F').reshape(3,edge_df.shape[0])

    rand = (np.random.rand(len(zs),3)*255).astype('int')
    rand[0] = 0
    cmap_rand = ListedColormap(rand)
    for i in range(len(zs)):
        fig.add_trace(
                go.Scatter3d(
                    x=[xs[i], xt[i]],
                    y=[ys[i], yt[i]],
                    z=[zs[i], zt[i]],
                    mode='lines',
                    line={"color":rand[[i,i,i]],
                          "width":10,
                         },                
                )
            )
        
    fig['layout'].update(scene=dict(aspectmode="manual", 
                                aspectratio=dict(x=X_SIZE*X_SHAPE, 
                                                 y=Y_SIZE*Y_SHAPE, 
                                                 z=Z_SIZE*Z_SHAPE)),
                     showlegend=False)
    fig.show()

In [129]:
import random 
if DIRECT_VISUALISATION_FACE:
    rand = (np.random.rand(face_df.shape[0],3)*255).astype('int')
    rand[0] = 0
    cmap_rand = ListedColormap(rand)


    plot_face=[]
    
    fill_colors=list(mpl.colors.cnames.values())
    random.shuffle(fill_colors)
    for f in (np.unique(edge_df.face)[1:]):

        try:
            edges= edge_df[edge_df.face==f]
            nb_vert = len(edges)
            vert_order = [edges.iloc[0].srce]
            for i in range(nb_vert):
                vert_order.append(edges[edges.trgt==vert_order[-1]]['srce'].to_numpy()[0])
            zs=vert_df.loc[vert_order].z_pix.to_numpy()
            ys=vert_df.loc[vert_order].y_pix.to_numpy()
            xs=vert_df.loc[vert_order].x_pix.to_numpy()
            plot_face.append(dict(
                type='scatter3d',
                mode='lines',
                x=xs,
                y=ys,
                z=zs,
                name='',
                surfaceaxis=2, # add a surface axis ('1' refers to axes[1] i.e. the y-axis)
                surfacecolor=fill_colors[f],
                line=dict(
                    color='black',
                    width=4
                ),
            ))
        except:
            print(f)



    plot_face.append(background_skeleton)
    plot_face.append(vertex)
    fig = go.Figure(data=plot_face, )
    fig.update_layout(title='Filament', 
                      autosize=False,
                      width=1000,
                      height=1000,
                      margin=dict(l=65, r=50, b=65, t=90),
                     )

    fig['layout'].update(scene=dict(aspectmode="manual", 
                                aspectratio=dict(x=X_SIZE*X_SHAPE, 
                                                 y=Y_SIZE*Y_SHAPE, 
                                                 z=Z_SIZE*Z_SHAPE)),
                     showlegend=False)
    fig.show()

In [130]:
import random 
if DIRECT_VISUALISATION_FACE:
    rand = (np.random.rand(face_df.shape[0],3)*255).astype('int')
    rand[0] = 0
    cmap_rand = ListedColormap(rand)


    plot_face=[]
    
    fill_colors=list(mpl.colors.cnames.values())
    random.shuffle(fill_colors)
    for f in (np.unique(edge_df.face)[10:12]):

        try:
            edges= edge_df[edge_df.face==f]
            nb_vert = len(edges)
            vert_order = [edges.iloc[0].srce]
            for i in range(nb_vert):
                vert_order.append(edges[edges.trgt==vert_order[-1]]['srce'].to_numpy()[0])
            zs=vert_df.loc[vert_order].z_pix.to_numpy()
            ys=vert_df.loc[vert_order].y_pix.to_numpy()
            xs=vert_df.loc[vert_order].x_pix.to_numpy()
            plot_face.append(dict(
                type='scatter3d',
                mode='lines',
                x=xs,
                y=ys,
                z=zs,
                name='',
                surfaceaxis=2, # add a surface axis ('1' refers to axes[1] i.e. the y-axis)
                surfacecolor=fill_colors[f],
                line=dict(
                    color='black',
                    width=4
                ),
            ))
        except:
            print(f)



    plot_face.append(background_skeleton)
    plot_face.append(vertex)
    fig = go.Figure(data=plot_face, )
    fig.update_layout(title='Filament', 
                      autosize=False,
                      width=1000,
                      height=1000,
                      margin=dict(l=65, r=50, b=65, t=90),
                     )

    fig['layout'].update(scene=dict(aspectmode="manual", 
                                aspectratio=dict(x=X_SIZE*X_SHAPE, 
                                                 y=Y_SIZE*Y_SHAPE, 
                                                 z=Z_SIZE*Z_SHAPE)),
                     showlegend=False)
    fig.show()

In [133]:
face_df.loc[10:11][['orient_x','orient_y','orient_z']]*180/np.pi

,orient_x,orient_y,orient_z
10,3.714212,109.599659,-51.518910
11,15.755233,106.307102,-14.099793


In [ ]:
face_df.loc[75]

# Analyse

In [132]:
# morphometry analysis

if MORPHOLOGICAL_ANALYSIS :
    morphology_analysis(face_df, 
                        edge_df, 
                        vert_df, 
                        points_df, 
                        area=AREA,
                        perimeter=PERIMETER,
                        nb_neighbor=NB_NEIGHBOR,
                        aniso=ANISO,
                       )

In [ ]:
i = 55
plt.plot(edge_df.loc[i]['point_x'][0], edge_df.loc[i]['point_y'][0], '.')

## Mesure d'intensité au niveau des jonctions

In [ ]:

# import image original
# get original image

img_myosin = io.imread(os.path.join(directory, 
                              "C1-20171214_sqh-GFP_ap-alpha-cat-RFP_WP2h-001-dorsal_AiSc_green_bin4.tif"))


junction_intensity(img_myosin, edge_df, 'myosin_intensity')

In [ ]:
# plot prot intensity

import plotly.graph_objects as go

z0, y0,x0 = np.where(img_binary_3d==1)
fond = go.Scatter3d(x=x0,
                      y=y0,
                      z=z0,
                      mode='markers',
                     marker=dict(
                        size=2,
                        color='black',
                        opacity=0.1
                     )
    )


vertex_isolated = go.Scatter3d(x=vert_df.x,
                      y=vert_df.y,
                      z=vert_df.z,
                      mode='markers',
                     marker=dict(
                        size=2,
                        color='red', 
                        opacity=1
                     )
    )



fig = go.Figure(data=[vertex_isolated], )
fig.update_layout(title='Filament', 
                  autosize=False,
                  width=1000,
                  height=1000,
                  margin=dict(l=65, r=50, b=65, t=90),
                 )



zs,ys,xs = vert_df.loc[edge_df['srce']][list('zyx')].values.flatten(order='F').reshape(3, edge_df.shape[0])
zt,yt,xt = vert_df.loc[edge_df['trgt']][list('zyx')].values.flatten(order='F').reshape(3, edge_df.shape[0])
c = (edge_df.myosin_intensity/np.max(edge_df.myosin_intensity)*255).astype('int')

import pylab as pl
cmap = pl.cm.coolwarm


for i in range(len(zs)):
    
    fig.add_trace(
            go.Scatter3d(
                x=[xs[i], xt[i]],
                y=[ys[i], yt[i]],
                z=[zs[i], zt[i]],
                mode='lines',
                line={"color":[cmap(c[i]), cmap(c[i])],
                      "width":10,
                     },                
            )
        )


fig['layout'].update(scene=dict(aspectmode="data"), showlegend=False)
fig.show()

In [ ]:
# plot area

import plotly.graph_objects as go

import pylab as pl
cmap = pl.cm.cividis
# face_df['area']-=face_df.area.min()
# area=face_df.area.values/np.max(face_df.area.values)*255
plot_face=[]
for f in (np.unique(edge_df.face)[1:]):
#     print('rgb'+str(cmap(int(ar[f]))))
    edges= edge_df[edge_df.face==f]
    nb_vert = len(edges)
    vert_order = [edges.iloc[0].srce]
    for i in range(nb_vert):
        vert_order.append(edges[edges.trgt==vert_order[-1]]['srce'].to_numpy()[0])
    zs=vert_df.loc[vert_order].z.to_numpy()
    ys=vert_df.loc[vert_order].y.to_numpy()
    xs=vert_df.loc[vert_order].x.to_numpy()
    plot_face.append(dict(
        type='scatter3d',
        mode='lines',
        x=xs,
        y=ys,
        z=zs,
        name='',
        surfaceaxis=2, # add a surface axis ('1' refers to axes[1] i.e. the y-axis)
        surfacecolor='rgb'+str(cmap(int(face_df['area'].iloc[f-1]))[0:3]),
#             surfacecolor=fill_colors[int(ar[f]/2)],
        line=dict(
            color='black',
            width=4
        ),
    ))
   
    
fig = go.Figure(data=plot_face, )
fig.update_layout(title='Filament', 
                  autosize=False,
                  width=1000,
                  height=1000,
                  margin=dict(l=65, r=50, b=65, t=90),
                 )

fig['layout'].update(scene=dict(aspectmode="data"), showlegend=False)
fig.show()

In [ ]:
fsgdfgdg

# Analyse au niveau de la face

In [ ]:
from Dissects.analysis.analysis_3D_apical import face_intensity, compute_normal

In [ ]:
# define pixel size [x,y,z] in µm
pixel_size = {'x':0.1832311, 'y':0.1832311, 'z':0.2201818}
# Largeur d'épaississement de la surface de la cellule
# Cette valeur est ajoutée au dessus et en dessous du plan de la face
thicken = 0.4
# compute_normal(face_df, edge_df, vert_df)
face_intensity(img_myosin, face_df, edge_df, vert_df, thicken, pixel_size)


In [ ]:
# plot area

import plotly.graph_objects as go

import pylab as pl
cmap = pl.cm.cividis
face_df['intensity']-=face_df.intensity.min()
face_df['intensity_norm']=face_df.intensity.values/np.max(face_df.intensity.values)*255
plot_face=[]
for f in (np.unique(edge_df.face)[1:]):
#     print('rgb'+str(cmap(int(ar[f]))))
    edges= edge_df[edge_df.face==f]
    nb_vert = len(edges)
    vert_order = [edges.iloc[0].srce]
    for i in range(nb_vert):
        vert_order.append(edges[edges.trgt==vert_order[-1]]['srce'].to_numpy()[0])
    zs=vert_df.loc[vert_order].z.to_numpy()
    ys=vert_df.loc[vert_order].y.to_numpy()
    xs=vert_df.loc[vert_order].x.to_numpy()
    plot_face.append(dict(
        type='scatter3d',
        mode='lines',
        x=xs,
        y=ys,
        z=zs,
        name='',
        surfaceaxis=2, # add a surface axis ('1' refers to axes[1] i.e. the y-axis)
        surfacecolor='rgb'+str(cmap(int(face_df['intensity_norm'].iloc[f-1]))[0:3]),
#             surfacecolor=fill_colors[int(ar[f]/2)],
        line=dict(
            color='black',
            width=4
        ),
    ))
   
    
fig = go.Figure(data=plot_face, )
fig.update_layout(title='Filament', 
                  autosize=False,
                  width=1000,
                  height=1000,
                  margin=dict(l=65, r=50, b=65, t=90),
                 )

fig['layout'].update(scene=dict(aspectmode="data"), showlegend=False)
fig.show()

In [ ]:
import plotly.graph_objects as go
from matplotlib.colors import ListedColormap
rand = (np.random.rand(200,3)*255).astype('int')
rand[0] = 0
cmap_rand = ListedColormap(rand)



z0, y0,x0 = np.where(img_binary_3d==1)
fond = go.Scatter3d(x=x0,
                      y=y0,
                      z=z0,
                      mode='markers',
                     marker=dict(
                        size=2,
                        color='black',
                        opacity=1
                     )
    )

z0, y0,x0 = np.where(img_plane==1)
face = go.Scatter3d(x=x0,
                      y=y0,
                      z=z0,
                      mode='markers',
                     marker=dict(
                        size=2,
                        color='blue', 
                        opacity=0.2
                     )
    )


plot_face=[]
plot_face.append(fond)
plot_face.append(face)
fig = go.Figure(data=plot_face, )
fig.update_layout(title='Filament', 
                  autosize=False,
                  width=1000,
                  height=1000,
                  margin=dict(l=65, r=50, b=65, t=90),
                 )

fig['layout'].update(scene=dict(aspectmode="data"), showlegend=False)
fig.show()

In [ ]:
from tyssue import Sheet
from tyssue import SheetGeometry
face_df = pd.DataFrame(index=np.arange(1, np.max(edge_df.face)))
# remove_edge = edge_df[edge_df.face==-1].index
# edge_df.drop(remove_edge, axis=0, inplace=True)
# edge_df.reset_index(inplace=True, drop=True)
data={'vert':vert_df, 'edge':edge_df, 'face':face_df}
sheet = Sheet('spherical', data)
sheet.reset_index()
sheet.reset_topo()
SheetGeometry.update_all(sheet)

from tyssue.draw import sheet_view
fig, ax = sheet_view(sheet,list('yz'), mode='2D')

In [ ]:
from tyssue.draw import sheet_view
fig, ax = sheet_view(sheet, mode='3D')
fig